<a href="https://colab.research.google.com/github/Youngdeuk-Koo/KNLP/blob/main/%EB%B3%B4%EC%BA%A1_%EB%A7%8C%EB%93%A4%EA%B8%B0(%EC%84%BC%ED%85%90%EC%8A%A4%ED%94%BC%EC%8A%A4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
%cd ../

!pip install sentencepiece

     |████████████████████████████████| 19.4 MB 4.3 MB/s 
     |████████████████████████████████| 448 kB 68.5 MB/s 
Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 109 (delta 7), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (109/109), 1.27 MiB | 5.53 MiB/s, done.
Resolving deltas: 100% (46/46), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2022-02-22 08:17:49--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::3403:4be7, 2406:da00:ff00::22c5:2ef4, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

In [2]:
korea_path = '/content/drive/MyDrive/AI_Study/NLP/DATA/한국_일본 text/clean_ko_corpus.txt'
japan_path = '/content/drive/MyDrive/AI_Study/NLP/DATA/한국_일본 text/clean_ja_corpus.txt'

with open(korea_path, 'r', encoding='utf-8') as f:
    korea = f.read().splitlines()

with open(japan_path, 'r', encoding='utf-8') as f:
    japan = f.read().splitlines()

In [3]:
from konlpy.tag import Mecab

mecab = Mecab()

k_corpus = [mecab.morphs(w) for w in korea]
j_corpus = [mecab.morphs(w) for w in japan]

k_flatten_corpus = [w for s in k_corpus for w in s]
j_flatten_corpus = [w for s in j_corpus for w in s]

In [4]:
import sentencepiece as spm

def generate_tokenizer(corpus,
                       vocab_size,
                       name=None,
                       pad_id = 0,
                       bos_id = 1,
                       eos_id = 2,
                       unk_id = 3):    

    file = f'./{name}_corpus.txt'
    model = f'{name}_spm'

    with open(file, 'w', encoding='utf-8') as f:
        for row in corpus: f.write(str(row) + '\n')

    spm.SentencePieceTrainer.Train(
        f'--input=./{file} --model_prefix={model} --vocab_size={vocab_size + 4}' +
        ' --model_type=bpe' +
        ' --max_sentence_length=999999' +
        f' --pad_id={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id}'
        # ' --user_defined_symbols=[SEP],[CLS],[MASK]'
    )

    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load(f'{model}.model')

    return tokenizer

In [5]:
VOCAB_SIZE = 100000
k_tokenizer = generate_tokenizer(k_flatten_corpus, VOCAB_SIZE, name='korea')

In [9]:
VOCAB_SIZE = 200000
j_tokenizer = generate_tokenizer(j_flatten_corpus, VOCAB_SIZE, name='japan')

In [11]:
vocab_file = '/content/korea_spm.model'
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "겨울이 되어서 날씨가 무척 추워요.",
  "이번 성탄절은 화이트 크리스마스가 될까요?",
  "겨울에 감기 조심하시고 행복한 연말 되세요."
]

for line in lines:
    pieces = vocab.encode_as_pieces(line)
    ids = vocab.encode_as_ids(line)
    print(line)
    print(pieces)
    print(ids)

겨울이 되어서 날씨가 무척 추워요.
['▁겨울', '이', '▁되', '어', '서', '▁날씨', '가', '▁무척', '▁추워', '요', '.']
[1670, 98781, 49, 98815, 98796, 3807, 98787, 8707, 21143, 98915, 98783]
이번 성탄절은 화이트 크리스마스가 될까요?
['▁이번', '▁성탄절', '은', '▁화이트', '▁크리스마스', '가', '▁될까요', '?']
[335, 32444, 98798, 6728, 8250, 98787, 51874, 99645]
겨울에 감기 조심하시고 행복한 연말 되세요.
['▁겨울', '에', '▁감기', '▁조심', '하', '시고', '▁행복', '한', '▁연말', '▁되', '세요', '.']
[1670, 98785, 5307, 5736, 98790, 45665, 1420, 98793, 2801, 49, 25026, 98783]


In [10]:
vocab_file = '/content/japan_spm.model'
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

lines = [
  "私の人生の中で一番大事な事だった。",
  "明日の天気は晴れなの？",
  "家族と旅行に行って楽しい時間を過ごしました。"
]

for line in lines:
    pieces = vocab.encode_as_pieces(line)
    ids = vocab.encode_as_ids(line)
    print(line)
    print(pieces)
    print(ids)

私の人生の中で一番大事な事だった。
['▁私', 'の', '人生', 'の', '中', 'で', '一番', '大', '事', 'な', '事', 'だった', '。']
[478, 197521, 185309, 197521, 197589, 197531, 120055, 197583, 197608, 197537, 197608, 2119, 197530]
明日の天気は晴れなの？
['▁明日', 'の', '天', '気', 'は', '晴', 'れな', 'の', '?']
[5610, 197521, 198267, 197720, 197534, 199416, 25945, 197521, 198996]
家族と旅行に行って楽しい時間を過ごしました。
['▁家族', 'と', '旅行', 'に', '行', 'って', '楽', 'しい', '時間', 'を', '過', 'ご', 'しました', '。']
[1062, 197529, 56886, 197523, 197588, 55, 198107, 1493, 10388, 197526, 197848, 197952, 7634, 197530]
